In [1]:
import numpy as np  # NumPy for arrays
import torch  # PyTorch for tensors
import torch.nn as nn  # Neural network module
import matplotlib.pyplot as plt  # For plotting
from matplotlib import cm  # Colormap for plots

In [2]:
class NN(nn.Module):
    def __init__(self,
                 dim_hidden = 128,
                layers = 1,
                hidden_bias = True):
        super().__init__()
        self.dim_hidden= dim_hidden
        self.layers = layers
        self.hidden_bias = hidden_bias

        torch.manual_seed(123)
        module = []
        module.append(nn.Linear(1,self.dim_hidden, bias = self.hidden_bias))
        module.append(nn.Tanh())

        for i in range(self.layers-1):
            module.append(nn.Linear(self.dim_hidden,self.dim_hidden, bias = self.hidden_bias))
            module.append(nn.Tanh())

        module.append(nn.Linear(self.dim_hidden,1))
        module.append(nn.Softplus(beta = 1.0)) #The softplus layer ensures c>0,k>0

        self.q = nn.Sequential(*module)


    def forward(self, x):
        out = self.q(x) # first element is consumption, the second element is capital
        return  out

In [3]:
v_nn = NN()

In [4]:
a = 0
b = 3
n = 5 # number of Gauss Legendre nodes

In [5]:
def int_quad_gl_tensor(v, a, b, n):
    # v: the neural network (or any callable)
    # a: lower bound of the integral
    # b: upper bound of the integral
    # n: number of the nodes
    nodes, weights = np.polynomial.legendre.leggauss(n)
    nodes_tensor = torch.tensor(nodes, dtype=torch.float32).unsqueeze(-1)
    weights_tensor = torch.tensor(weights, dtype=torch.float32).unsqueeze(-1)
    adjusted_nodes = ((b - a) / 2) * nodes_tensor + ((a + b) / 2)
    approximation = ((b - a) / 2) * torch.sum(weights_tensor * v(adjusted_nodes))
    return approximation

In [6]:
gauss_Legendre_quad_int = int_quad_gl_tensor(v = v_nn, a = 0, b = 3, n = 5).item()
gauss_Legendre_quad_int

1.0490837097167969

In [7]:
def int_LLN(v,a,b,N):
    x_samples = (b - a) * torch.rand(N, 1) + a
    v_values = v(x_samples)
    integral = (b-a)*v_values.mean()
    return integral

In [8]:
# Making the animation

In [9]:
# grid number of draws N
# Building the number of nodes array
# 1) 1 to 200 step 1
N_1 = np.arange(1, 200, 1)

# 2) 200 to 1000 step 10
N_2 = np.arange(200, 1001, 10)

# 3) 1000 to 5000 step 50
N_3 = np.arange(1000, 10001, 30)

# Combine all parts
N_grid = np.unique(np.concatenate([N_1, N_2, N_3]))

In [10]:
integral_values = []
for N in N_grid:
    integral_v = int_LLN(v = v_nn,a = a, b = b, N = N).item()
    integral_values.append(integral_v)

integral_values = np.array(integral_values)

In [11]:
fontsize= 14
ticksize = 14
figsize = (9, 6)
params = {'font.family':'serif',
    "figure.figsize":figsize,
    'figure.dpi': 80,
    'figure.edgecolor': 'k',
    'font.size': fontsize,
    'axes.labelsize': fontsize,
    'axes.titlesize': fontsize,
    'xtick.labelsize': ticksize,
    'ytick.labelsize': ticksize
}
plt.rcParams.update(params)

In [12]:
import os
import numpy as np
import matplotlib.pyplot as plt
import imageio.v2 as imageio  # use v2 API to avoid warning

In [13]:
filenames = []

for i in range(N_grid.shape[0]):
    plt.axhline(y=gauss_Legendre_quad_int, color='red', linestyle='--', label='Gauss–Legendre quadrature: 5 nodes')
    plt.plot(N_grid[:i], integral_values[:i], color='k',label="Monte Carlo approximation")
    plt.xlim(1, N_grid.max()+1)
    plt.ylim(integral_values.min()-0.1, integral_values.max()+0.3)
    plt.xscale('log')
    plt.text(10, 1.48, f"Number of draws: {N_grid[i]}", ha='center', fontsize=18)
    plt.text(10, 1.37, "$\int_{a}^b v(x) dx$", ha='center', fontsize=18)
    plt.xlabel(r"Number of draws ($N$)")
    plt.title(r"Monte Carlo vs Gauss–Legendre Quadrature for Neural Network Integration", fontsize=14)
    plt.legend()
    plt.tight_layout()
    
    filename = f'{i}.png'
    filenames.append(filename)

    # save frame
    plt.savefig(filename)
    plt.close()# build gif
with imageio.get_writer('Gauss–Legendre.gif', mode='I') as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)
        
# Remove files
for filename in set(filenames):
    os.remove(filename)

